In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None # display all the columns

# Feautre data

In [2]:
df_original = pd.read_csv('Data_features.csv') # feature set

# Factual Data for Training and User ID List for Prediction

In [3]:
obs = pd.read_csv('Data_train_set.csv') # User ID, Churn
est = pd.read_csv('Data_test_set.csv')  # User ID to predict Churn

In [4]:
results_sample = pd.read_csv('Sample_test.txt')
results_sample

,ID,TARGET
0,1,0.420288
1,3,0.449351
2,6,0.465414
3,7,0.166205
4,12,0.645974
5,18,0.023114


# Analyze some specific cases

In [5]:
len(obs[obs.TARGET == 1])/len(obs) * 100

8.08378588052754

8% will Churn within next 3 month.

In [6]:
obs_id = obs['ID'].values # IDs in factual data
est_id = est['ID'].values # IDs to analyse
len(obs_id) + len(est_id) == 17195

True

Make a dataframe for training, i.e. select entries with known outcome

In [7]:
df = df_original[df_original.ID.isin(obs_id)] # df for training
len(df)/6 == 12890

True

I will use ```df_obs``` to train the model, then I will make prediction about ```est.ID```.

Entries with IDs of people who will churn

In [8]:
df_churn = df[df.ID.isin(obs.ID[obs['TARGET']==1])] # entries of people who churn 
df_churn[0:6]

,ID,MONTH_NUM_FROM_EVENT,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F26,F27,F28,F29,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39,F40,F41,F42,F43,F44,F45,F46,F47
83,21,1,10.0,56.0,27.0,1.0,0.0,223.77,201.73,15.33,74.60,63.22,11.38,173.0,NaN,7.0,60.0,49.0,11.0,17.86,0.0,25.36,0.189501,0.0,NaN,0.0,19.0,25.0,28.0,30.0,0.0,0.0,1.0,2.0,40.0,72.0,84.0,3.0,79.0,134.0,2.0,3.0,3.0,2.0,0.0,NaN,2.0,393.0
84,21,2,25.0,55.0,26.0,3.0,NaN,335.83,324.65,9.75,108.97,108.03,0.93,158.0,133.0,NaN,84.0,82.0,2.0,NaN,8.0,59.64,18.318316,0.0,NaN,0.0,25.0,26.0,27.0,29.0,0.0,0.0,2.0,2.0,40.0,100.0,62.0,3.0,16.0,138.0,4.0,5.0,5.0,2.0,0.0,0.0,4.0,534.0
85,21,3,25.0,54.0,30.0,0.0,0.0,292.10,287.70,NaN,104.22,100.38,0.17,117.0,111.0,3.0,49.0,46.0,1.0,39.50,0.0,40.00,0.000000,0.0,NaN,0.0,29.0,29.0,30.0,NaN,0.0,0.0,3.0,2.0,40.0,56.0,40.0,2.0,7.0,134.0,3.0,4.0,4.0,2.0,0.0,0.0,3.0,272.0
86,21,4,25.0,NaN,30.0,0.0,0.0,139.05,137.47,1.47,82.88,69.87,13.02,118.0,111.0,2.0,74.0,67.0,7.0,43.90,0.0,47.50,0.000000,0.0,NaN,0.0,25.0,28.0,30.0,31.0,0.0,0.0,4.0,2.0,40.0,NaN,NaN,NaN,8.0,NaN,4.0,4.0,4.0,1.0,0.0,0.0,1.0,367.0
87,21,5,25.0,52.0,27.0,0.0,0.0,110.60,110.60,0.00,70.78,53.77,3.05,64.0,64.0,0.0,38.0,35.0,1.0,46.50,0.0,46.50,0.000000,0.0,NaN,0.0,23.0,26.0,28.0,29.0,0.0,NaN,5.0,NaN,40.0,NaN,NaN,2.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,21,6,NaN,NaN,25.0,10.0,0.0,247.33,243.70,3.37,43.03,36.62,0.00,114.0,106.0,NaN,37.0,33.0,0.0,63.50,0.0,92.50,93.957795,0.0,NaN,0.0,18.0,21.0,24.0,25.0,0.0,0.0,6.0,2.0,40.0,NaN,NaN,2.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


I would like to make a table of all the 13k users with all the 47x6 features.

But before I would like to make some predictions based on the last month usage.

In [9]:
df_lm = df[df.MONTH_NUM_FROM_EVENT == 1] # the last month data
df_lm.head() # notice that df doesn't include column F25

,ID,MONTH_NUM_FROM_EVENT,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F26,F27,F28,F29,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39,F40,F41,F42,F43,F44,F45,F46,F47
0,2,1,4.0,133.0,28.0,NaN,0.0,NaN,518.98,2.60,151.33,142.32,9.02,168.0,155.0,6.0,84.0,71.0,13.0,NaN,0.0,56.48,377.331409,0.000000,NaN,0.0,0.0,0.0,0.0,19.0,0.0,0.0,1.0,1.0,40.0,99.0,75.0,5.0,89.0,137.0,0.0,2.0,2.0,1.0,0.0,0.0,2.0,958.0
6,4,1,5.0,139.0,29.0,7.0,0.0,682.90,676.53,3.53,591.27,585.22,6.05,174.0,164.0,4.0,258.0,253.0,5.0,46.50,0.0,49.25,25.455420,13.795844,NaN,0.0,0.0,0.0,0.0,5.0,0.0,0.0,1.0,NaN,NaN,267.0,72.0,14.0,NaN,130.0,2.0,2.0,4.0,0.0,2.0,0.0,1.0,866.0
12,5,1,20.0,19.0,22.0,0.0,1.0,137.98,55.78,36.03,115.33,28.90,41.55,47.0,15.0,21.0,45.0,26.0,15.0,0.00,0.0,128.00,0.000000,0.000000,NaN,0.0,0.0,2.0,4.0,9.0,1.0,0.0,NaN,NaN,NaN,NaN,22.0,5.0,44.0,12.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,146.0
18,8,1,21.0,134.0,29.0,21.0,1.0,459.90,408.58,11.92,485.28,413.35,25.42,155.0,114.0,7.0,95.0,80.0,6.0,10.85,8.0,123.40,8.501169,NaN,NaN,0.0,1.0,21.0,22.0,25.0,0.0,0.0,NaN,3.0,40.0,103.0,35.0,1.0,59.0,137.0,2.0,2.0,2.0,NaN,NaN,NaN,1.0,1201.0
24,9,1,20.0,9.0,28.0,28.0,0.0,813.60,628.88,181.68,391.65,321.33,70.32,540.0,363.0,175.0,353.0,284.0,69.0,469.62,0.0,549.32,25841.457227,25845.000255,NaN,0.0,0.0,0.0,2.0,4.0,16.0,4.0,1.0,5.0,66.0,513.0,285.0,9.0,182.0,142.0,0.0,1.0,4.0,0.0,1.0,1.0,2.0,34688.0


In [10]:
X = np.array(df_lm.values)[:,2:] # array of features from 
X.shape

(12890, 46)

In [11]:
y = np.array(obs.TARGET)
y.shape

(12890,)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

In [13]:
from sklearn.utils.validation import assert_all_finite

nanplaces = np.isnan(X_train)
X_train[nanplaces] = 0
np.any(np.isnan(X_train))

nanplaces = np.isnan(X_test)
X_test[nanplaces] = 0
np.any(np.isnan(X_test))

# run checks

print(assert_all_finite(X_train))
print(assert_all_finite(X_test))
# should output None
print(np.any(np.isnan(X_train)), np.all(np.isfinite(X_train))) 
print(np.any(np.isnan(X_test)), np.all(np.isfinite(X_test)))
# should output False True



None
None
False True
False True


In [14]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# k Nearest Neighbors Classifier

In [15]:
from sklearn.neighbors import KNeighborsClassifier

k = 3
# Train Model and Predict

neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)

y_ = neigh.predict(X_test)
match = y_ == y_test

print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, y_))
print("AUC ROC score", metrics.roc_auc_score(y_test, y_))
print(confusion_matrix(y_test, y_.round()))


Train set Accuracy:  0.9289177657098526
Test set Accuracy:  0.908844065166796
AUC ROC score 0.5199570627876783
[[2331   50]
 [ 185   12]]


From the confusion matrix we can see that KNN doesn't work too...

# Multiple Linear Regression

In [16]:
from sklearn.linear_model import LinearRegression

# Train Model and Predict

regr = LinearRegression().fit(X_train,y_train)

y_ = regr.predict(X_test)
match = y_ == y_test

In [17]:
print("AUC ROC score", metrics.roc_auc_score(y_test, y_))

AUC ROC score 0.7642376086488423


In [18]:
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train).round()))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, y_.round()))
print(sum(y_test == y_.round()), 'cases correctly guessed, out of', len(y_)) 

Train set Accuracy:  0.9289177657098526
Test set Accuracy:  0.9235841737781226
2381 cases correctly guessed, out of 2578


In [19]:
ybin_ = np.where(y_>0.5,1,0)
confusion_matrix(y_test, ybin_)

array([[2381,    0],
       [ 197,    0]])

In [20]:
comp = y_test != y_.round() # see mistakes

In [21]:
itemindex = np.where(comp==True)
itemindex

(array([  11,   12,   29,   58,   73,  103,  115,  117,  132,  172,  176,
         196,  200,  211,  217,  228,  241,  272,  287,  289,  318,  332,
         347,  399,  401,  406,  416,  450,  482,  492,  494,  523,  524,
         562,  563,  564,  565,  585,  608,  609,  622,  647,  682,  700,
         709,  727,  738,  758,  767,  768,  779,  786,  792,  821,  831,
         856,  886,  888,  890,  911,  925,  936,  939,  947,  972,  974,
         975,  992, 1012, 1015, 1034, 1044, 1052, 1064, 1069, 1077, 1082,
        1088, 1104, 1117, 1120, 1121, 1122, 1125, 1169, 1177, 1180, 1190,
        1214, 1237, 1241, 1243, 1259, 1262, 1290, 1300, 1312, 1349, 1352,
        1363, 1377, 1388, 1400, 1407, 1427, 1429, 1435, 1451, 1457, 1458,
        1464, 1481, 1529, 1539, 1542, 1543, 1577, 1579, 1584, 1587, 1614,
        1622, 1628, 1636, 1643, 1657, 1661, 1669, 1670, 1672, 1684, 1688,
        1690, 1698, 1699, 1745, 1763, 1818, 1850, 1853, 1860, 1869, 1876,
        1883, 1886, 1887, 1901, 1921, 

In [22]:
[
    [y_test[12], y_[12]], # a mistake!
    [y_test[2502], y_[2502]], # too
]  

[[1.0, 0.13966153614344126], [1.0, 0.22069578019160271]]

> Maybe I should lower threshold (replace ```round()``` with ```1 if p>0.4 else 0```)...

Well, linear regression misses all churn... __open__...<!-- The score of the model that says there is NO people who churn is slighly less than 0.8 -that may be a lie-->

# Logistic Regression

We normalize the dataset:

In [23]:
from sklearn.utils.validation import assert_all_finite

nanplaces = np.isnan(X)
X[nanplaces] = 0

# run checks
print(assert_all_finite(X))
# should output None
print(np.any(np.isnan(X)), np.all(np.isfinite(X))) 
# should output False True

None
False True


In [24]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (10312, 46) (10312,)
Test set: (2578, 46) (2578,)


In [26]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)

In [27]:
y_prob_ = LR.predict_proba(X_test)[:,1]
y_ = LR.predict(X_test)
y_prob_.shape, y_.shape

((2578,), (2578,))

In [28]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test, y_)

0.9208688906128782

In [29]:
confusion_matrix(y_test, y_, labels=[1,0])

array([[   2,  204],
       [   0, 2372]])

In [30]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_))

             precision    recall  f1-score   support

        0.0       0.92      1.00      0.96      2372
        1.0       1.00      0.01      0.02       206

avg / total       0.93      0.92      0.88      2578



In [31]:
print("AUC ROC score", metrics.roc_auc_score(y_test, y_))

AUC ROC score 0.5048543689320388


In [32]:
print("AUC ROC score", metrics.roc_auc_score(y_test, y_prob_))

AUC ROC score 0.7559308436614877


In [33]:
from sklearn.metrics import log_loss
log_loss(y_test, y_prob_)

0.25282770149405903

In [34]:
sum(y_test)/len(y_test), len(y_test)

(0.07990690457719161, 2578)

In [35]:
print("AUC ROC score", metrics.roc_auc_score(y_test, np.zeros(2578)))
log_loss(y_test, np.zeros(2578))

AUC ROC score 0.5


2.759886709601087